In [ ]:
# code source : https://github.com/braingeneers/SIMS/blob/main/scsims/model.py

In [ ]:
from functools import partial
from typing import Any, Callable, Dict, Union

import os
import anndata as an
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from pytorch_tabnet.tab_network import TabNet
from pytorch_tabnet.utils import create_explain_matrix
from scipy.sparse import csc_matrix
from torchmetrics.functional.classification.stat_scores import _stat_scores_update
from tqdm import tqdm
import torch.utils.data
from scipy.sparse import csr_matrix
from scsims.data import CollateLoader
from scsims.inference import DatasetForInference
from scsims.temperature_scaling import _ECELoss
from torchmetrics import Accuracy, F1Score, Precision, Recall, Specificity
from sklearn.preprocessing import LabelEncoder

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class SIMSClassifier(pl.LightningModule):
    def __init__(
        self,
        input_dim,
        output_dim,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        cat_idxs=[],
        cat_dims=[],
        cat_emb_dim=1,
        n_independent=2,
        n_shared=2,
        epsilon=1e-15,
        virtual_batch_size=128,
        momentum=0.02,
        mask_type="sparsemax",
        lambda_sparse=1e-3,
        optim_params: Dict[str, float] = None,
        scheduler_params: Dict[str, float] = None,
        weights: torch.Tensor = None,
        loss: Callable = None,  # will default to cross_entropy
        pretrained: bool = None,
        no_explain: bool = False,
        genes: list[str] = None,
        cells: list[str] = None,
        label_encoder: LabelEncoder = None,
        *args,
        **kwargs,
    ) -> None:
        super().__init__()
        self.save_hyperparameters()
        self.genes = genes
        self.label_encoder = label_encoder

        # Stuff needed for training
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.lambda_sparse = lambda_sparse
        self.optim_params = optim_params
        self.weights = weights
        self.loss = loss

        if self.loss is None:
            self.loss = F.cross_entropy

        if pretrained is not None:
            self._from_pretrained(**pretrained.get_params())

        self.metrics = {
            "train": {x: y.to(device) for x, y in aggregate_metrics(num_classes=self.output_dim).items()},
            "val": {x: y.to(device) for x, y in aggregate_metrics(num_classes=self.output_dim).items()},
        }

        self.optim_params = (
            optim_params
            if optim_params is not None
            else {
                "optimizer": torch.optim.Adam,
                "lr": 0.01,
                "weight_decay": 0.01,
            }
        )

        self.scheduler_params = (
            scheduler_params
            if scheduler_params is not None
            else {
                "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau,
                "factor": 0.75,  # Reduce LR by 25% on plateau
            }
        )

        print(f"Initializing network")
        self.network = TabNet(
            input_dim=input_dim,
            output_dim=output_dim,
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            cat_idxs=cat_idxs,
            cat_dims=cat_dims,
            cat_emb_dim=cat_emb_dim,
            n_independent=n_independent,
            n_shared=n_shared,
            epsilon=epsilon,
            virtual_batch_size=virtual_batch_size,
            momentum=momentum,
            mask_type=mask_type,
        )

        print(f"Initializing explain matrix")
        if not no_explain:
            self.reducing_matrix = create_explain_matrix(
                self.network.input_dim,
                self.network.cat_emb_dim,
                self.network.cat_idxs,
                self.network.post_embed_dim,
            )

        self._inference_device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.temperature = torch.nn.Parameter(torch.ones(1) * 1.5)

    def forward(self, x):
        logits, M_loss = self.network(x)
        # temp scaling will be 1 so logits wont change until model is calibrated
        return self.temperature_scale(logits), M_loss

    def _step(self, batch, tag):
        x, y = batch
        logits, M_loss = self.network(x)

        loss = self.loss(logits, y, weight=self.weights)
        loss = loss - self.lambda_sparse * M_loss

        # take softmax for metrics
        probs = logits.softmax(dim=-1)

        # if binary, probs will be (batch, 2), so take second column
        if probs.shape[-1] == 2:
            probs = probs[:, 1]

        tp, fp, _, fn = _stat_scores_update(
            preds=logits,
            target=y,
            num_classes=self.output_dim,
            reduce="macro",
        )

        return {
            "loss": loss,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "probs": probs,
        }

    # Calculations on step
    def training_step(self, batch, batch_idx):
        results = self._step(batch, "train")
        self.log(f"train_loss", results["loss"], on_epoch=True, on_step=True)
        for name, metric in self.metrics["train"].items():
            value = metric(results["probs"], batch[1])
            self.log(f"train_{name}", value=value)

        return results["loss"]

    def on_train_epoch_end(self) -> None:
        for name, metric in self.metrics["train"].items():
            value = metric.compute()
            self.log(f"train_{name}", value=value)
            metric.reset() # inplace

    def validation_step(self, batch, batch_idx):
        results = self._step(batch, "val")
        self.log(f"val_loss", results["loss"], on_epoch=True, on_step=True)
        for name, metric in self.metrics["val"].items():
            value = metric(results["probs"], batch[1])
            self.log(f"val_{name}", value=value)

        return results["loss"]

    def on_validation_epoch_end(self) -> None:
        for name, metric in self.metrics["val"].items():
            value = metric.compute()
            self.log(f"val_{name}", value=value)
            metric.reset()

    def configure_optimizers(self):
        if "optimizer" in self.optim_params:
            optimizer = self.optim_params.pop("optimizer")
            optimizer = optimizer(self.parameters(), **self.optim_params)
        else:
            optimizer = torch.optim.Adam(self.parameters(), **self.optim_params)
        print(f"Initializing with {optimizer = }")

        if self.scheduler_params is not None:
            scheduler = self.scheduler_params.pop("scheduler")
            scheduler = scheduler(optimizer, **self.scheduler_params)
            print(f"Initializating with {scheduler = }")

        if self.scheduler_params is None:
            return optimizer

        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "train_loss",
        }

    def _parse_data(
            self,
            inference_data,
            batch_size=64,
            num_workers=os.cpu_count(),
            rows=None,
            **kwargs
        ) -> torch.utils.data.DataLoader:

        if isinstance(inference_data, str):
            inference_data = an.read_h5ad(inference_data)
        
        # handle zero inflation or deletion
        #TODO: Increase speed and memory consumption of zero inflation
        inference_genes = list(inference_data.var_names)
        training_genes = list(self.genes)

        # more inference genes than training genes
        assert len(set(inference_genes).intersection(set(training_genes))) > 0, "inference data shares zero genes with training data, double check the string formats and gene names"

        left_genes = list(set(inference_genes) - set(training_genes))  # genes in inference that aren't in training
        right_genes = list(set(training_genes) - set(inference_genes))  # genes in training that aren't in inference 
        intersection_genes = list(set(inference_genes).intersection(set(training_genes))) # genes in both

        if len(left_genes) > 0:
            inference_data = inference_data[:, intersection_genes].copy()
        if len(right_genes) > 0:
            print(f"Inference data has {len(right_genes)} less genes than training; performing zero inflation.")

            #zero_inflation = an.AnnData(X=np.zeros((inference_data.shape[0], len(right_genes))), obs=inference_data.obs)
            zero_inflation = an.AnnData(X= csr_matrix((inference_data.shape[0], len(right_genes))),obs=inference_data.obs)
            zero_inflation.var_names = right_genes
            inference_data = an.concat([zero_inflation, inference_data], axis=1)

        # now make sure the columns are the correct order
        inference_data = inference_data[:, training_genes].copy()

        if isinstance(inference_data, an.AnnData):
            inference_data = DatasetForInference(inference_data.X[rows, :] if rows is not None else inference_data.X)

        if not isinstance(inference_data, torch.utils.data.DataLoader):
            inference_data = CollateLoader(
                dataset=inference_data,
                batch_size=batch_size,
                num_workers=num_workers,
                **kwargs,
            )

        return inference_data

    def explain(
        self,
        anndata,
        rows=None,
        batch_size=64,
        num_workers=os.cpu_count(),
        currgenes=None,
        refgenes=None,
        normalize=False,
        **kwargs,
    ) -> tuple[np.ndarray, np.ndarray]:
        loader = self._parse_data(anndata, batch_size=batch_size, num_workers=num_workers, rows=rows, currgenes=currgenes, refgenes=refgenes, **kwargs)

        self.network.eval()
        res_explain = []

        all_labels = np.empty(len(loader.dataset))
        all_labels[:] = np.nan

        for batch_nb, data in enumerate(tqdm(loader)):
            # if we are running this on already labeled pairs and not just for inference
            if isinstance(data, tuple):
                X, label = data
                all_labels[batch_nb * batch_size : (batch_nb + 1) * batch_size] = label
            else:
                X = data.float()

            M_explain, masks = self.network.forward_masks(X)
            for key, value in masks.items():
                masks[key] = csc_matrix.dot(
                    value.cpu().detach().numpy(), self.reducing_matrix
                )

            original_feat_explain = csc_matrix.dot(M_explain.cpu().detach().numpy(),
                                                   self.reducing_matrix)
            res_explain.append(original_feat_explain)

            if batch_nb == 0:
                res_masks = masks
            else:
                for key, value in masks.items():
                    res_masks[key] = np.vstack([res_masks[key], value])

        res_explain = np.vstack(res_explain)

        if normalize:
            res_explain /= np.sum(res_explain, axis=1)[:, None]

        return res_explain, all_labels

    def _compute_feature_importances(self, dataloader):
        M_explain, _ = self.explain(dataloader, normalize=False)
        sum_explain = M_explain.sum(axis=0)
        feature_importances_ = sum_explain / np.sum(sum_explain)

        return feature_importances_

    def feature_importances(self, dataloader, cache=False):
        if cache and self._feature_importances is not None:
            return self._feature_importances
        else:
            f = self._compute_feature_importances(dataloader)
            if cache:
                self._feature_importances = f
            return f

    def predict(self, inference_data: Union[str, an.AnnData, np.array], batch_size=32, num_workers=4, rows=None, currgenes=None, refgenes=None, **kwargs):
        print("Parsing inference data...")
        loader = self._parse_data(
            inference_data,
            batch_size=batch_size,
            num_workers=num_workers,
            rows=rows,
            currgenes=currgenes,
            refgenes=refgenes,
            **kwargs
        )

        # initialize arrays in memory and fill with nans to start
        # this makes it easier to see bugs/wrong predictions than filling zeros
        num_cls_to_save = min(3, len(self.label_encoder.classes_))
        preds = np.empty((len(loader.dataset), num_cls_to_save))
        preds[:] = np.nan

        all_labels = np.empty(len(loader.dataset))
        all_labels[:] = np.nan

        # save probs 
        probs = np.empty((len(loader.dataset), num_cls_to_save))
        probs[:] = np.nan

        prev_network_state = self.network.training
        self.network.eval()

        # batch size might differ if user passes in a dataloader
        batch_size = loader.batch_size
        for idx, X in enumerate(tqdm(loader)):
            # Some dataloaders will have all_labels, handle this case
            top_probs, top_preds, label = self.predict_step(batch=X, batch_idx=idx)
            all_labels[idx * batch_size : (idx + 1) * batch_size] = label
            preds[idx * batch_size : (idx + 1) * batch_size] = top_preds
            probs[idx * batch_size : (idx + 1) * batch_size] = top_probs

        preds = pd.DataFrame(preds).astype(int)

        preds = preds.rename(columns={i: f"pred_{i}" for i in range(preds.shape[1])})
        preds = preds.apply(lambda x: self.label_encoder.inverse_transform(x))

        probs = pd.DataFrame(probs)
        probs = probs.rename(columns={i: f"prob_{i}" for i in range(probs.shape[1])})

        final = pd.concat([preds, probs], axis=1)

        if not np.all(np.isnan(all_labels)):
            final["label"] = all_labels

        # if network was in training mode before inference, set it back to that
        if prev_network_state:
            self.network.train()

        return final

    def predict_step(self, batch: Any, batch_idx: int, dataloader_idx: int = 0) -> Any:
        if len(batch) == 2:
            data, label = batch
        else:
            data, label = batch, None
        data = data.float()
        res = self(data)[0]
        num_sample = min(len(self.label_encoder.classes_), 3)
        probs, top_preds = res.topk(num_sample, axis=1)  # to get indices
        probs = probs.softmax(dim=-1)

        return probs.detach().cpu().numpy(), top_preds.detach().cpu().numpy(), label
 
    def temperature_scale(self, logits):
        """
        Perform temperature scaling on logits
        """
        # Expand temperature to match the size of logits
        temperature = self.temperature.unsqueeze(1).expand(logits.size(0), logits.size(1))  # (Batch, Classes)
        return logits / temperature  # (Batch, Classes)

    def set_temperature(self, dataloader, max_iter=50, lr=0.01):
        """
        Tune the temperature of the model (using the validation set).
        We're going to set it to optimize NLL.
        dataloader (DataLoader): validation set loader
        """
        nll_criterion = torch.nn.CrossEntropyLoss()
        ece_criterion = _ECELoss()

        # First: collect all the logits and labels for the validation set
        logits_list = []
        labels_list = []
        print("Setting temperature ...")
        with torch.no_grad():
            for data, label in tqdm(dataloader):
                logits = self(data)[0]
                logits_list.append(logits)
                labels_list.append(label)
            logits = torch.cat(logits_list) # (num_samples*batch_size, num_classes)
            labels = torch.cat(labels_list)

        # Calculate NLL and ECE before temperature scaling
        before_temperature_nll = nll_criterion(logits, labels).item()
        before_temperature_ece = ece_criterion(logits, labels).item()
        print('Before temperature - NLL: %.3f, ECE: %.3f' % (before_temperature_nll, before_temperature_ece))

        # Next: optimize the temperature w.r.t. NLL
        optimizer = torch.optim.LBFGS([self.temperature], lr=lr, max_iter=max_iter)

        def eval():
            optimizer.zero_grad()
            loss = nll_criterion(self.temperature_scale(logits), labels)
            loss.backward()
            return loss

        optimizer.step(eval)
        
        after_temperature_nll = nll_criterion(self.temperature_scale(logits), labels).item()
        after_temperature_ece = ece_criterion(self.temperature_scale(logits), labels).item()
        print('Optimal temperature: %.3f' % self.temperature.item())
        print('After temperature - NLL: %.3f, ECE: %.3f' % (after_temperature_nll, after_temperature_ece))

        return self

def confusion_matrix(model, dataloader, num_classes):
    confusion_matrix = torch.zeros(num_classes, num_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(tqdm(dataloader)):
            outputs, _ = model(inputs)

            _, preds = torch.max(outputs, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

    return confusion_matrix


def median_f1(tps, fps, fns):
    precisions = tps / (tps + fps)
    recalls = tps / (tps + fns)

    f1s = 2 * (np.dot(precisions, recalls)) / (precisions + recalls)

    return np.nanmedian(f1s)


def aggregate_metrics(num_classes) -> Dict[str, Callable]:
    task = "binary" if num_classes == 2 else "multiclass"
    num_classes = None if num_classes == 2 else num_classes

    metrics = {
        "micro_accuracy": Accuracy(task=task, num_classes=num_classes, average="micro"),
        "macro_accuracy": Accuracy(task=task, num_classes=num_classes, average="macro"),
        "weighted_accuracy": Accuracy(task=task, num_classes=num_classes, average="weighted"),
        "precision": Precision(task=task, num_classes=num_classes, average="macro"),
        "recall": Recall(task=task, num_classes=num_classes, average="macro"),
        "f1": F1Score(task=task, num_classes=num_classes, average="macro"),
        "specificity": Specificity(task=task, num_classes=num_classes, average="macro"),
    }

    return metrics

这些代码实现了一个基于**TabNet**的深度学习模型，主要用于分类任务。TabNet 是一种面向表格数据的深度学习模型，特别适合处理结构化数据。该代码利用了**PyTorch Lightning**来简化模型的训练和验证过程。以下是代码的详细介绍，包括模型架构、训练过程、推理过程等。

### 1. **TabNet 网络架构**
`TabNet` 是一种专门为表格数据（如基因表达矩阵、拉曼光谱数据等）设计的深度学习网络。它的主要特点是通过注意力机制学习特征的重要性。TabNet 的架构包括以下几个关键组件：

- **Input Embedding (输入嵌入)**：通过神经网络对输入特征进行变换。
- **Attention Mechanism (注意力机制)**：TabNet 使用注意力机制在不同的输入特征之间进行动态加权，从而决定哪些特征在当前时刻最为重要。
- **Decision Tree Blocks**：由多层决策树构成，用于根据当前输入选择合适的特征。
- **Sparsity Constraint (稀疏性约束)**：为了减少过拟合，TabNet 强制一些特征在每一轮训练中保持稀疏。

### 2. **SIMSClassifier 模型**
`SIMSClassifier` 是一个继承自 `pl.LightningModule` 的类，用于定义和训练模型。这个模型结合了 `TabNet` 和 PyTorch Lightning。以下是其组成部分：

#### 初始化方法 (`__init__`)
- **模型参数**：包括输入维度、输出维度、TabNet的超参数等。`TabNet` 的超参数如 `n_d`, `n_a`, `n_steps` 等影响网络的学习过程。
- **损失函数**：如果没有传入损失函数，则使用交叉熵损失（`F.cross_entropy`）。
- **优化器和调度器**：初始化 Adam 优化器以及学习率调度器（如 `ReduceLROnPlateau`）。

#### 前向传播 (`forward`)
- 前向传播包括 `TabNet` 模型的执行，其中输出 `logits` 和 `M_loss`，`logits` 是分类的概率分布，`M_loss` 是稀疏性损失（对特征稀疏性的约束）。
- 使用 `temperature_scale` 方法对 logits 进行温度缩放，从而调整模型输出的温度。

#### 训练和验证步骤 (`training_step` 和 `validation_step`)
- 通过 `training_step` 和 `validation_step` 方法，模型计算损失并更新其权重。
- `training_step` 会记录训练中的损失和多个指标（如准确率、精度等）。
- `on_train_epoch_end` 和 `on_validation_epoch_end` 在每个 epoch 结束时汇总训练和验证阶段的结果。

#### 优化器和调度器 (`configure_optimizers`)
- 定义优化器（`Adam`）以及学习率调度器（`ReduceLROnPlateau`），以便在训练过程中调整学习率。

#### 推理 (`predict` 和 `predict_step`)
- `predict` 方法用于模型推理，接受原始数据并生成预测。
- `predict_step` 对每一个批次的输入数据执行推理并返回概率和预测标签。

#### 解释性方法 (`explain` 和 `feature_importances`)
- `explain` 方法利用 `TabNet` 的解释性模块生成特征重要性矩阵，以帮助理解哪些特征对模型的预测起到了决定性作用。
- `feature_importances` 方法计算每个特征的相对重要性。

### 3. **温度缩放（Temperature Scaling）**
温度缩放是用来校准分类模型输出的概率分布的一种方法。它通过一个可学习的温度参数 `temperature` 对模型的输出 logits 进行缩放，优化使得模型的负对数似然损失（NLL）最小化，从而提高模型的预测概率的可靠性。

### 4. **推理数据处理**
在 `SIMSClassifier` 中，推理数据会通过 `_parse_data` 方法处理，该方法确保输入数据的格式和训练数据一致。如果推理数据的特征比训练数据少，系统会使用零膨胀（zero inflation）技术，通过为缺失的特征添加零来填补数据。

### 5. **评估指标**
在训练和验证阶段，使用了多个评估指标来衡量模型的性能，包括：
- **Accuracy**（准确率）
- **Precision**（精度）
- **Recall**（召回率）
- **F1 Score**
- **Specificity**（特异性）

这些指标使用 `torchmetrics` 库计算，能够提供不同维度的性能评价。

### 6. **稀疏性损失（Sparse Loss）**
在 `TabNet` 中，为了提高模型的可解释性，网络中每一层会加入稀疏性损失（`M_loss`），通过强制某些特征的注意力值为零来实现。这个稀疏性约束的系数由 `lambda_sparse` 控制。

### 总结
整体上，`SIMSClassifier` 是基于 **TabNet** 的深度学习模型，专门设计用于处理表格型数据（如基因表达数据或拉曼光谱数据）。通过使用 **PyTorch Lightning** 来管理训练和推理流程，模型支持稀疏性约束、温度缩放以及特征重要性计算等功能。此外，模型的输出可以进行温度校准，以便更准确地调整分类结果的概率分布。

TabNet 和普通的 **MLP（多层感知机）** 在结构和工作原理上有几个重要区别，尤其是在处理表格数据（如结构化数据）时。下面是两者之间的主要差异：

### 1. **处理方式**
- **MLP**：
  - **传统的全连接层**：MLP 是由多个全连接层（`Linear`）构成，每个层都通过激活函数（如 ReLU）进行非线性变换。
  - **输入特征处理**：所有输入特征通常都被处理为固定维度的向量，网络不具备处理特征重要性的机制。所有特征的影响是等同的，除非人为地在特征工程过程中加以处理。
  - **特征选择**：特征选择和特征重要性的评估通常需要通过外部方法（如决策树、L1 正则化等）来进行。

- **TabNet**：
  - **自适应特征选择**：TabNet 使用了一种基于注意力机制的方法来自动选择输入数据中的重要特征。每一层中的注意力机制动态地决定哪些特征在当前步骤中最为重要。这意味着模型在每个决策步骤中都能自适应地选择最相关的特征，而不需要人工进行特征选择。
  - **稀疏性约束**：TabNet 强制特征的选择过程具有稀疏性，减少了不重要特征的参与，从而增强了模型的可解释性和性能。
  - **解释性**：由于 TabNet 强调特征选择的透明度，模型能够为每个预测提供更清晰的解释，即哪些特征在决策中起了主导作用。

### 2. **网络结构**
- **MLP**：
  - 由一系列的 **全连接层（Fully Connected Layers）** 组成，输入层与隐藏层之间、隐藏层之间，以及隐藏层与输出层之间都进行加权求和，然后通过非线性激活函数进行映射。
  - 每一层的输出是所有输入特征的加权和，通常没有专门的机制去优化特征选择或强调某些特征。
  - **参数量较大**，尤其是输入特征数量较多时，可能会导致过拟合或计算开销较高。

- **TabNet**：
  - 由多个 **决策树块（Decision Blocks）** 组成，每个块内部使用了 **注意力机制** 来学习输入特征的加权。
  - **输入特征的选择是逐步进行的**，每个决策步骤通过一个注意力机制来计算一个稀疏的特征选择分布，使用该分布来决定输入中哪些特征对下一步计算最为重要。
  - 采用了 **稀疏激活机制**，通过控制每一层选择的特征数，使得每个决策块都能够聚焦于少数几个重要特征，而不需要使用所有输入特征。

### 3. **特征处理**
- **MLP**：
  - 特征输入通常是经过预处理的标准化或归一化数据，所有特征以相同的方式输入并参与计算。
  - MLP 没有特征选择的机制，每个特征都会对每一层的输出产生影响。
  
- **TabNet**：
  - **自动特征选择**：TabNet 通过注意力机制选择输入特征的子集，并在每一层自适应地调整这些特征的权重。
  - 通过 **嵌入特征（Embedding）** 和 **稀疏注意力**，模型能够更加灵活地处理不同类型的特征（例如连续型和类别型特征）。

### 4. **注意力机制**
- **MLP**：
  - 不使用注意力机制，所有输入特征都是平等参与计算的。
  
- **TabNet**：
  - **基于注意力的特征选择**：每一层都会根据当前的输入和先前的历史决策，学习一个稀疏的注意力分布，动态选择一部分特征进行计算。这样，网络会逐步聚焦于最相关的特征，而不是同时使用所有特征。
  - **特征的重要性在训练过程中动态更新**，这一机制有助于模型发现最有信息量的特征。

### 5. **稀疏性**
- **MLP**：
  - MLP 中的每一层都会接收所有输入特征的加权和，因此并没有稀疏性机制。所有特征都对结果产生一定影响，除非通过正则化（如 L1 正则化）强制某些特征的系数变为零。

- **TabNet**：
  - **稀疏激活机制**：每一层使用注意力机制对特征进行选择并加权，进而限制了每一层使用的特征数量。这种稀疏性机制能够防止过拟合，并提高模型的解释性，因为每一步决策的特征选择是透明的。

### 6. **训练过程**
- **MLP**：
  - 训练过程通常较为直接，依赖标准的梯度下降优化算法（如 Adam、SGD）来优化网络参数。

- **TabNet**：
  - TabNet 训练过程中不仅优化模型的参数，还需要通过**温度缩放**、**稀疏性约束**等技术来优化注意力机制的效果。模型的训练更注重对特征选择和稀疏激活的控制，以保证更高效的学习。

### 7. **可解释性**
- **MLP**：
  - MLP 的可解释性较差。由于其全连接的结构，特征的权重和影响是混合在一起的，较难直观地解释模型的决策过程。

- **TabNet**：
  - TabNet 提供了较好的可解释性。通过注意力机制，模型可以清楚地展示哪些特征在每一轮决策中起到了主导作用。这对于许多实际应用（尤其是需要可解释性的行业，如医疗、金融）非常重要。

### 总结
- **MLP** 是一种基础的神经网络架构，适合处理结构化数据，但它的处理方式较为简单，特征选择和解释性较差。
- **TabNet** 则通过引入注意力机制、稀疏激活和特征选择机制，能够在处理结构化数据时更有效地选择和使用特征，且具有更好的可解释性和性能。

在处理复杂的表格数据时，TabNet 通常会表现得比 MLP 更好，尤其是在特征维度较高时。